# NIRCam Coronagraphy plotting notebook

Author: Jonathan Aguilar (jaguilar@stsci.edu)

This notebook filters the enormous number of NIRCam aperture options to just the ones relevant for cornagraphy. For a more general introduction to WHIPPOT, see [WHIPPOT_tutorial](WHIPPOT_tutorial).

The user provides the RA and Dec coordinates for the target acquisition (ACQ) star and science (SCI) star, and a telescope roll angle. These tools will then compute a) the "Special Requirment Offset" X and Y values needed to place the SCI star at the reference position, and b) the final coordinates of the SCI and ACQ stars. Coordinates are given in IDL (https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-target-observability-and-observatory-coordinate-system/jwst-instrument-ideal-coordinate-systems#gsc.tab=0).

The user can also provide a list of extra targets in the field in the format specified below, and their IDL coordinates will also be computed. As such, this notebook is useful not just for planning target acquisition, but also for predicting to high precision the final positions of astronomical bodies in the field of view.

## Setup ##

- With installing the module:
  1. Navigate in a terminal to the directory where you have downloaded the module. This should contain the `setup.py` file.
  2. Activate your desired python environment.
  3. Run `pip install .`
  4. Copy this notebook to your working directory, and run your jupyter kernel in the appropriate environment.
- Without installing the module:
  1. Copy this notebook and `utils.py` to your working directory.
  2. In the cell below, change `from all_jwst_offset_ta import utils` to `import utils`
  3. Open the notebook as you normally would. The following python modules must be installed in the working enviroment:
     -  `ipywidgets`
     - `numpy`
     - `matplotlib`
     - `astropy`
     - `pysiaf`

In [1]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [2]:
# for your system, choose an appropriate plotting backend to make pop-up window plots
# (currently, inline plots show up in the log but not directly in the notebook).
# %matplotlib throws an error if you run it first, so do inline first.
%matplotlib inline
%matplotlib osx

In [3]:
from whippot import whippot_tools
from whippot.modes import nircam_coron_tools
# if the library is not installed replace the line above with the line below 
# when `whippot_tools.py` is in the same folder as the notebook
# import whippot_tools

## An example of a real field

Let's take the HR 8799 system as an example. WHIPPOT requires RA and Dec coordinates for all sources, but the positions of imaged planets are usually given relative to the host star. We can use astropy's handy SkyCoord object to make the conversion simple. We'll get the offsets from http://whereistheplanet.com/ as delta RA and delta Dec, but SkyCoord can also work with separation and position angle.

In [4]:
from astropy import units
primary = whippot_tools.SkyCoord("23 07 28.7157209544 +21 08 03.310767492", frame='icrs', unit=("hourangle","degree")) # from SIMBAD
sources = {
    'SCI': primary,
    'b': primary.spherical_offsets_by(1648.876*units.mas, 487.424*units.mas),
    'c': primary.spherical_offsets_by(-226.103*units.mas, 930.851 *units.mas),
    'd': primary.spherical_offsets_by(-640.700*units.mas, -284.250*units.mas),
    'e': primary.spherical_offsets_by(-152.636*units.mas, 376.193*units.mas),
}

In [5]:
# format the sources for easy copy-pasting
for s, c in sources.items():
    if s != 'SCI':
        print(f"{s}: ({c.ra.deg}, {c.dec.deg})")

b: (346.870139887803, 21.13438838581789)
c: (346.8695815016935, 21.134511560678916)
d: (346.86945803121176, 21.13417403252982)
e: (346.8696033809097, 21.134357489019493)


In [6]:
from astropy import units
initial_values={
    'instr': 'nircam',
    'sci_aper': 'nrca5_mask335r', 
    'pa': 290.,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
    'exclude_roi': True,
}
# add a multi-line string of the other stars, copied from the cell above
initial_values['other_stars'] = "\n".join(f"{k}: ({v.ra.deg}, {v.dec.deg})" for k, v in sources.items() if k not in ['SCI'])

## Use the NIRCam Coronagraphy Tools module

We have subclassed the ComputePositions() object from `whippot_tools` (see the `WHIPPOT_tutorial.ipynb` notebook) and modified it to better suit NIRCam users.

In [24]:
cp1 = nircam_coron_tools.ComputePositions(initial_values=initial_values)
cp1.ui

In [11]:
a = 'adafd'
a.count('d')

2

In [15]:
aperture_list = list(cp1.instr.apernames)
apernames = list(filter(lambda i: i.startswith("NIRCA"), aperture_list))


In [20]:
list(filter(lambda f: f > 1, [-1, 0, 1, 2, 3]))

[2, 3]

In [21]:
aperture_list

['NRCA1_FULL_OSS',
 'NRCA2_FULL_OSS',
 'NRCA3_FULL_OSS',
 'NRCA4_FULL_OSS',
 'NRCA5_FULL_OSS',
 'NRCB1_FULL_OSS',
 'NRCB2_FULL_OSS',
 'NRCB3_FULL_OSS',
 'NRCB4_FULL_OSS',
 'NRCB5_FULL_OSS',
 'NRCALL_FULL',
 'NRCAS_FULL',
 'NRCA1_FULL',
 'NRCA2_FULL',
 'NRCA3_FULL',
 'NRCA4_FULL',
 'NRCA5_FULL',
 'NRCBS_FULL',
 'NRCB1_FULL',
 'NRCB2_FULL',
 'NRCB3_FULL',
 'NRCB4_FULL',
 'NRCB5_FULL',
 'NRCA1_SUB160',
 'NRCA2_SUB160',
 'NRCA3_SUB160',
 'NRCA4_SUB160',
 'NRCA5_SUB160',
 'NRCA1_SUB320',
 'NRCA2_SUB320',
 'NRCA3_SUB320',
 'NRCA4_SUB320',
 'NRCA5_SUB320',
 'NRCA1_SUB640',
 'NRCA2_SUB640',
 'NRCA3_SUB640',
 'NRCA4_SUB640',
 'NRCA5_SUB640',
 'NRCB1_SUB160',
 'NRCB2_SUB160',
 'NRCB3_SUB160',
 'NRCB4_SUB160',
 'NRCB5_SUB160',
 'NRCB1_SUB320',
 'NRCB2_SUB320',
 'NRCB3_SUB320',
 'NRCB4_SUB320',
 'NRCB5_SUB320',
 'NRCB1_SUB640',
 'NRCB2_SUB640',
 'NRCB3_SUB640',
 'NRCB4_SUB640',
 'NRCB5_SUB640',
 'NRCA5_GRISM256_F322W2',
 'NRCA5_GRISM128_F322W2',
 'NRCA5_GRISM64_F322W2',
 'NRCA5_GRISM256_F277W',
 '